# Predictions of fraudlent users in a Bitcoin user network #

On platforms where users interact with each other to perform some type of financial transaction, it is very desirable to stop users who act in a fraudulent manner from being able to access or continue to use the platform. However, investigating users can be very expensive (consider e.g sellers on Ebay or Amazon) or impossible (consider users on a Bitcoin network) depending on the context. It is therefore worth asking the following - is it possible to perform accurate inference for the trustworthiness of users simply based on their behavior on the platform? We investigate this using publically available data for a Bitcoin user platform, where we observe ratings between users (and the times at which these ratings are made), and only know for a small subset of users whether they are trustworthy or not. 

## Summary of data ## 

For the dataset we are considering, we have 5,881 nodes (corresponding to separate users) and 35,592 weighted edges (corresponding to ratings from one user to another), along with time-stamp data; for example, a subset of the data looks like the following:

Source node | Target node | Rating | Time stamp
:----------:|:-----------:|:------:|:---------:
6 | 2 |	4 | 1289241912
6 | 5 | 2 | 1289241942
1 | 15 | 1 | 1289243140
4 | 3 | 7 | 1289245277
13 | 16 | 8 | 1289254254
13 | 10 | 8 | 1289254301
7 | 5 | 1 | 1289362700
2 | 21 | 5 | 1289370557

We note that ratings are integer valued between -10 to 10 (going from not-trustworthy to trustworthy) - when we analyse the data, we will normalize these to lie in \[0, 1\], and we will ignore the fact that user ratings are given on an integer scale. This is done mostly for computational reasons - it will be easier to try and infer a likelihood/probability of trustworthiness using variational inference or MCMC methods than trying to perform inference for the discrete latent state. If we were really interested in performing inference for the underlying latent state, then we could simply do so by thresholding these probabilities at a later state. The measurements for the time stamps are given as measurements of the time in seconds after 'Epoch' (January 1st 1970) in which the ratings were made; although we could try and use this information more substanatially (and e.g try and model trustworthiness as a function of time), for now we will only use it to compute per-node summary statistics. 

We also have some notion of "ground truth" data for the users in the network - there are 136 users which are considered trustworthy, and 180 (so we have ground truth data for 316 users in total). Note that this is only for a subset of the users, so we will use a semi-supervised approach to modelling our data. As our end goal is to make inference for the trustworthiness for users, we will split the ground truth data in the following way:

* We randomly select 200 user ratings to use as a training set;
* We keep the remaining 116 to use as a test set.

Why split the ground truth data? Note that we only have partial information as to whether users are trustworthy, yet our overall goal is to infer some statistic of trustworthiness for **every** user. We therefore keep some of our ground truth data to the side so we can use it to ensure that our models are correctly calibrated when trying to make inference for the user trustworthiness. 

## A quick caveat on our notion of ground truth ## 
Given that our data is taken from an inherently anonymous platform, you may (rightfully) ask whether the notion of 'ground-truth trustworthiness' comes from. For this dataset, the platform founder and those who they rated highly are considered to be trustworthy, and the users rated very lowly by the latter group are considered to be un-trustworthy. (In terms of the numerical specifics, I'm afraid I don't have those at hand - it's not very clear from the data source what exactly was done to obtain this information.) As a result, this is a clear limitation as to whether these users are actually acting fraudulently or not - instead we are really making inference for whether the platform holder would consider someone to be trustworthy. From e.g the perspective of the platform founder, this probably would suffice. 

# Exploratory data analysis

Before beginning to model the data in order to try and make inference for the trustworthiness of different users, we begin by exploring some of the features of our dataset to see what could potentially be informative for the purposes of achieving our overall goal. In particular, we're going to examine the following statistics of the data:

* The in and out degree distribution (i.e, how many ratings a user has given and recieved)
* The distribution of user ratings, and how users rate other users 
* The standard deviation of the time at which ratings are made
* Rating asymmetry - the difference between the in and out degrees of a node, and simiarly the difference of the average in rating and out rating

For the users whose ground truth state is available, we'll highlight their distribution in blue/red (corresponding to trustworthy or not-trustworthy), and otherwise keep the overall distribution in black/grey. 

# Stochastic Black Box Variational Inference for Network Data

Recall that in variational inference, the goal is to approximate a desired posterior distribution $p(\mathbf{z} | \mathbf{x})$ of some latent variables $\mathbf{z}$ given our data $\mathbf{x}$ using a variational family $q(\mathbf{z} | \lambda)$, with $\lambda$ parameterizing our variational family. To try and do so in an automated fashion, we update $\lambda$ via performing gradient ascent on the ELBO, whose gradient is 

$$ \nabla_{\lambda} \mathrm{ELBO} = \mathbb{E}_{\mathbf{z} \sim q(\cdot|\lambda) }\Big[ \nabla_{\lambda} \log q(\mathbf{z} | \lambda) \big\{ \log p(\mathbf{x}, \mathbf{z}) - \log q(\mathbf{z} | \lambda) \big\} \Big]   $$

In practice, we then usually make use of the two following ideas in order to perform computationally efficient inference:

* **Approximating the expectation** - We draw samples from $\mathbf{z} \sim q(\cdot | \lambda)$ to give a Monte Carlo estimate of the above expectation;
* **Approximating the log-likelihood** - To illustrate through an example, suppose we are in a regime where the likelihood for the data factors as $p(\mathbf{x} | \mathbf{z}) = \prod_{i=1}^N p(x_i | z_i)$ - i.e we have $N$ data points $x_i$ which are conditionally independent given local latent variables $z_i$. If our number of data points $N$ is large, rather than compute $\sum_{i=1}^N \log p(x_i | z_i) + \log(z_i)$, we instead draw a subset $I_M$ of $M$ data points and instead use the approximation
$$ \sum_{i=1}^N \big\{ \log p(x_i | z_i) + \log(z_i) \big\} \approx \frac{N}{M} \sum_{i \in I_M} \big\{ \log p(x_i | z_i) + \log(z_i) \big\}. $$
This works well in practice even when $M \ll N$, i.e when the number of subsampled data points is far smaller than the total number. 

Focusing on the latter idea for now, this approach of subsampling data points is very intuitive for when we are fitting data which either naturally arises as being conditionally independent (such as for data we may consider modelling through a traditional regression), or we can extract some statistics from our data which can then be modelled in the same way (consider for example a topic model given a set of documents, where all we use are counts of words from a group of documents). 

However, for certain types of structured data, some care and thought is required in trying to exploit ideas of subsampling to approximate the log-likelihood. Suppose we observe a undirected graph $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ where the number of nodes $|\mathcal{V}| = N$, and $\mathcal{E}$ is the list of edges; we can also equivalently think of having observed an adjacency matrix $A = (a_{ij})_{i, j \in \mathcal{V}}$. Some real-world examples of data collected in this format include:

* **Social networks** - Here, users of the social network correspond to the vertices, and edges correspond to whether they are e.g friends on the network. You can imagine that we also have nodal information corresponding to characteristics of each user; for example, age, gender, geographical location, hobbies/interests and et cetera.
* **Protein-protein interaction networks** - In these networks, we look at different proteins within a particular type of cell, and form a network with proteins as vertices, and edges denoting whether two proteins contribute to a shared biological function.
* **User-trust networks** - Here, users correspond to vertices, with an edge from one vertex to the other corresponding to a user providing a rating, the edge label corresponding to a rating of trust one user gave to another.

In the first two examples, we note that the presence of the edge between two vertices should be more likely if the two vertices have comparable latent features, and in the latter the distribution of the edge labels (that is, the ratings between users) should be different if a non-fraudlant user is rating a fruadlant user, as compared to a fraudlant user rating rating another fraudlant user. To summarize, for network data, while we care about making latent observations about each observed vertex, we need to do so using some amount of edge information. 

For such a network (supposing for now it is directed and without self-loops), we could imagine trying to form a Bayesian model of the form  

$$ \begin{aligned}
p(\omega_1, \ldots, \omega_N) &= \prod_{i=1}^N N(\omega_i \,|\, 0, \eta^2 I_d) \\
p(\{ a_{ij} \,|\, i \neq j \} \,|\, \omega_1, \ldots, \omega_n) &= \prod_{i \neq j} \mathrm{Bernoulli}\big(a_{ij} \,\big|\, \sigma(\langle \omega_i, \omega_j \rangle) \big)
\end{aligned}$$

where $\eta^2$ is specified in advance, and $\sigma(x) = e^x/(1+e^x)$ is the logistic function. We can then write the log-likelihood down as 

$$
p\big( \{ \omega_i \}_i, \{ a_{ij} \}_{i \neq j} \big) = \sum_{i \neq j} \Big\{ a_{ij} \log \sigma (\langle \omega_i, \omega_j \rangle) + (1 - a_{ij}) \log\big( 1 - \sigma(\langle u, v \rangle ) \big) \Big\} - \sum_{i=1}^n \frac{1}{2\sigma^2} \| \omega_i \|_2^2. 
$$

If we now let $S(\mathcal{G}) = (\mathcal{V}', \mathcal{E}')$ denoted a "subsampled" version of the network, where $\mathcal{V}' \subseteq \mathcal{V}$ and $\mathcal{E}'$ is a list of pairs of vertices $\mathcal{E}'$ (note that this may necessarily not be a subset of $\mathcal{E}$ of our original edge set), we could approximate the log-likelihood by 

$$
p\big( \{ \omega_i \}_i, \{ a_{ij} \}_{i \neq j} \big) \approx \frac{ N(N-1)  }{ |\mathcal{E}'| } \sum_{(i, j) \in \mathcal{E}'} \Big\{ a_{ij} \log \sigma (\langle \omega_i, \omega_j \rangle) + (1 - a_{ij}) \log\big( 1 - \sigma(\langle u, v \rangle ) \big) \Big\} - \frac{ N }{ |\mathcal{V}'|   } \sum_{i \in \mathcal{V}'} \frac{1}{2\sigma^2} \| \omega_i \|_2^2. 
$$

We now need to address the question of how we choose $S(\mathcal{G})$. Although there are some obvious first examples (e.g sample edges randomly), the work of Grover and Leskovec introduced "node2vec" [1], which uses biased random walks to help form $S(\mathcal{G})$. Later work by Vietch et al. [2] highlights that the **choice of sampling scheme is a modelling decision**, with some on-going work (D. and Austern, [3]) making explicit how changing the sampling scheme effects the latent structure recovered (under the assumption that the graph is exchangable). The important take-away from this is as follows:

> **For the purposes of Box's loop, we should take into account our choice of sampling scheme as part of our modeling step, moreso than as part of our inference step.**

One final remark: so far we've been talking implicitly about using variational inference to approximate the posterior distribution. Beyond the inability to use subsampling methods for MCMC methods to obtain samples from the posterior, there is one other thing to note: given the form of the likelihood and prior on the $\omega_i$ and $a_{ij}$ as above, the posterior will be _rotationally invariant_; that is, if $Q \in \mathbb{R}^{d \times d}$ is an orthogonal matrix, then 

$$ Q\omega_1, \ldots, Q \omega_N | a_{ij}, i \neq j \stackrel{\text{d}}{=} \omega_1, \ldots, \omega_N | a_{ij}, i \neq j. $$

As a consequence, the posterior mean (for example) of the $\omega_i$ will be equal to zero, and so extracting sensible summary statistics from our learned embeddings will be far more difficult. (For those familar with MCMC methods, this can be thought of as being due to a lack of identifiability of the $\omega_i$ in the above model, and so infact we would expect to have other computational difficulties in trying to sample from the posterior.) 

# References

[1] - 

[2] - 

[3] - 